## Problem Set 1
_MaCSS 222 Applied Statistics II_
_Spring 2025_

This Problem Set will utilize an extract from the 1997 cohort of the National Longitudinal Survey of Youth (NLSY97). This Notebook is designed to help you get started. Completing the problem set will involve adding code and commentary to this notebook. Narrative answers to questions posed in the Problem Set can be included in markdown boxes in this notebook. For "pencil and paper" calculations you can either hand-write your answers and turn in a pdf-scan of them along with your Python Jupyter Notebook, or you can write you answers in LaTex in markdown boxes. Please see the pdf file for Problem Set 1 for instructions.
<br>
<br>
If you are not already familiar with LaTex, I encourage you to learn the basics. Overleaf is a helpful online editing environment for LaTex which you can access as a UC Berkeley community member.

In [1]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part I: Analysis of NLSY97

In [2]:
# Directory where NLSY97 teaching extract file is located
data = '/Users/chris/Library/Mobile Documents/com~apple~CloudDocs/Berkeley/COMPSS 222 Applied Statistics II/Data/'

# Directory to save graphics files in
graphics = '/Users/chris/Library/Mobile Documents/com~apple~CloudDocs/Berkeley/COMPSS 222 Applied Statistics II/Graphics/'

In [3]:
# Load NLSY97 dataset
nlsy97 = pd.read_pickle(data + 'nlsy97.pkl')

In [4]:
# Rename some columns and then form new dataframe with complete cases for key variables
nlsy97.rename(columns={'avg_earn_2016_to_2020': 'earnings', 'hgc_at_age28': 'yrssch'}, inplace=True)
nlsy97 = nlsy97[['female','black','hispanic','yrssch','earnings']]
nlsy97 = nlsy97.dropna()

In [5]:
# Look at first few rows of dataset
nlsy97[0:5]

,,female,black,hispanic,yrssch,earnings
hhid97,pid97,,,,,
2,2,0,0,1,14.0,140764.964069
3,3,1,0,1,14.0,34487.266323
4,4,1,0,1,13.0,45864.738658
6,5,0,0,1,12.0,157108.595971
8,6,1,0,1,14.0,23202.102210


In [6]:
# Compute population share and mean earnings and variance for Black respondents
Pr_Black     = nlsy97['black'].mean()
E_earn_Black = nlsy97['earnings'].loc[nlsy97['black']==1].mean()
V_earn_Black = nlsy97['earnings'].loc[nlsy97['black']==1].var()

# Compute population share and mean earnings and variance for Hispanic respondents
Pr_Hispanic     = nlsy97['hispanic'].mean()
E_earn_Hispanic = nlsy97['earnings'].loc[nlsy97['hispanic']==1].mean()
V_earn_Hispanic = nlsy97['earnings'].loc[nlsy97['hispanic']==1].var()

# Compute population share and mean earnings and variance for white respondents
Pr_White     = 1 - Pr_Black - Pr_Hispanic
E_earn_White = nlsy97['earnings'].loc[(nlsy97['black']==0) & (nlsy97['hispanic']==0)].mean()
V_earn_White = nlsy97['earnings'].loc[(nlsy97['black']==0) & (nlsy97['hispanic']==0)].var()

# Compute mean earnings and variance for across all respondents
E_earn = nlsy97['earnings'].mean()
V_earn= nlsy97['earnings'].var()

# Collect calculations and put them in a dictionary
Earnings_Decomp = {'Race' : ['Black', 'Hispanic', 'White', 'Overall'], 'Prob' : [Pr_Black, Pr_Hispanic, Pr_White, 1], \
                   'E_Earn' : [E_earn_Black, E_earn_Hispanic, E_earn_White, E_earn], \
                   'V_Earn' : [V_earn_Black, V_earn_Hispanic, V_earn_White, V_earn]}

# Convert the dictionary to a DataFrame
Earnings_Decomp = pd.DataFrame(Earnings_Decomp)
Earnings_Decomp

,Race,Prob,E_Earn,V_Earn
0,Black,0.270709,38478.435225,1.924771e+09
1,Hispanic,0.212578,48728.665348,2.331829e+09
2,White,0.516713,63193.970507,3.914903e+09
3,Overall,1.000000,53428.240848,3.153443e+09


### 1.3 Data Cleaning and Preparation

In [8]:
num_records = len(nlsy97)
print(f"Number of complete cases: {num_records}")

Number of complete cases: 7569


In [27]:
# Compute summary statistics for numerical variables
summary_stats = nlsy97[['female', 'black', 'hispanic', 'yrssch', 'earnings']].describe()
summary_stats


,female,black,hispanic,yrssch,earnings
count,7569.000000,7569.000000,7569.000000,7569.000000,7569.000000
mean,0.501916,0.270709,0.212578,13.287885,53428.240848
std,0.500029,0.444356,0.409158,2.802136,56155.523140
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,12.000000,16265.219294
50%,1.000000,0.000000,0.000000,13.000000,41982.450082
75%,1.000000,1.000000,0.000000,16.000000,72244.145464
max,1.000000,1.000000,1.000000,20.000000,430919.831933


In [28]:
summary_stats = nlsy97.groupby("race_sex")["earnings"].describe()
summary_stats

,count,mean,std,min,25%,50%,75%,max
race_sex,,,,,,,,
Black Men,998.0,39923.698480,46957.851128,0.0,6951.648203,30393.791910,56261.943343,383978.885126
Black Women,1051.0,37106.053942,40701.499161,0.0,10205.604781,30336.138256,49696.123314,430919.831933
Hispanic Men,795.0,60248.847143,53780.672437,0.0,26700.208251,50666.811217,79172.374688,407449.358529
Hispanic Women,814.0,37477.382146,39124.697878,0.0,8539.642642,30458.973586,52600.624434,383978.885126
White Men,1977.0,78293.153793,68653.988955,0.0,36692.176871,63698.225708,97664.243937,430919.831933
White Women,1934.0,47759.076321,51275.386466,0.0,11308.535987,37282.210898,66329.532080,383978.885126


### 1.4 Sample Shares and Mean Earnings Calculation

In [16]:
# Compute the sample shares of each racial group
total_count = len(nlsy97)

# Sample shares
share_black = nlsy97['black'].mean()
share_hispanic = nlsy97['hispanic'].mean()
share_white = 1 - share_black - share_hispanic  # Assuming non-black, non-hispanic as white

# Compute mean earnings within each group
mean_earn_black = nlsy97.loc[nlsy97['black'] == 1, 'earnings'].mean()
mean_earn_hispanic = nlsy97.loc[nlsy97['hispanic'] == 1, 'earnings'].mean()
mean_earn_white = nlsy97.loc[(nlsy97['black'] == 0)&(nlsy97['hispanic'] == 0), 'earnings'].mean()

# Compute the unconditional mean of earnings using sample shares and group means
unconditional_mean_earnings = (
    share_black * mean_earn_black +
    share_hispanic * mean_earn_hispanic +
    share_white * mean_earn_white
)

# Compute the simple sample average of earnings
simple_sample_avg_earnings = nlsy97['earnings'].mean()

# Compare the computed mean with the simple sample average
comparison = {
    "Unconditional Mean Earnings": unconditional_mean_earnings,
    "Simple Sample Average Earnings": simple_sample_avg_earnings,
    "Difference": abs(unconditional_mean_earnings - simple_sample_avg_earnings)
}

# Display the results
comparison


{'Unconditional Mean Earnings': 53428.24084760573,
 'Simple Sample Average Earnings': 53428.24084760573,
 'Difference': 0.0}

### 1.5 Variance Decomposition by Race

In [17]:
# Compute conditional variances within each racial group
var_earn_black = nlsy97.loc[nlsy97['black'] == 1, 'earnings'].var()
var_earn_hispanic = nlsy97.loc[nlsy97['hispanic'] == 1, 'earnings'].var()
var_earn_white = nlsy97.loc[(nlsy97['black'] == 0) & (nlsy97['hispanic'] == 0), 'earnings'].var()

# Compute the overall variance of earnings
var_earn_total = nlsy97['earnings'].var()

# Compute within-group variance component
within_group_variance = (
    share_black * var_earn_black +
    share_hispanic * var_earn_hispanic +
    share_white * var_earn_white
)

# Compute between-group variance component
between_group_variance = (
    share_black * (mean_earn_black - simple_sample_avg_earnings) ** 2 +
    share_hispanic * (mean_earn_hispanic - simple_sample_avg_earnings) ** 2 +
    share_white * (mean_earn_white - simple_sample_avg_earnings) ** 2
)

# Verify decomposition
variance_decomposition_check = within_group_variance + between_group_variance

print("Variance Decomposition by Race:")
print(f"Total Variance: {var_earn_total:.2f}")
print(f"Within-Group Variance: {within_group_variance:.2f}")
print(f"Between-Group Variance: {between_group_variance:.2f}")
print(f"Check (Within + Between): {variance_decomposition_check:.2f}")

Variance Decomposition by Race:
Total Variance: 3153442779.11
Within-Group Variance: 3039629453.30
Between-Group Variance: 114476301.19
Check (Within + Between): 3154105754.49


### 1.6 Variance Decomposition by Gender

In [18]:
# Compute sample shares of female and male respondents
share_female = nlsy97['female'].mean()  # Proportion of females
share_male = 1 - share_female  # Proportion of males

# Compute mean earnings within each gender
mean_earn_female = nlsy97.loc[nlsy97['female'] == 1, 'earnings'].mean()
mean_earn_male = nlsy97.loc[nlsy97['female'] == 0, 'earnings'].mean()

# Compute conditional variance within each gender group
var_earn_female = nlsy97.loc[nlsy97['female'] == 1, 'earnings'].var()
var_earn_male = nlsy97.loc[nlsy97['female'] == 0, 'earnings'].var()

# Compute the overall variance of earnings
var_earn_total_gender = nlsy97['earnings'].var()

# Compute within-group variance component
within_group_variance_gender = (
    share_female * var_earn_female +
    share_male * var_earn_male
)

# Compute between-group variance component
between_group_variance_gender = (
    share_female * (mean_earn_female - simple_sample_avg_earnings) ** 2 +
    share_male * (mean_earn_male - simple_sample_avg_earnings) ** 2
)

# Verify decomposition
variance_decomposition_check_gender = within_group_variance_gender + between_group_variance_gender

print("Variance Decomposition by Gender:")
print(f"Total Variance: {var_earn_total_gender:.2f}")
print(f"Within-Group Variance: {within_group_variance_gender:.2f}")
print(f"Between-Group Variance: {between_group_variance_gender:.2f}")
print(f"Check (Within + Between): {variance_decomposition_check_gender:.2f}")

Variance Decomposition by Gender:
Total Variance: 3153442779.11
Within-Group Variance: 3035870046.78
Between-Group Variance: 117959190.47
Check (Within + Between): 3153829237.25


## Part II: Experimental design

In [19]:
import statsmodels.stats.power as smp

# Standard deviation of earnings from the dataset
std_earnings = nlsy97["earnings"].std()

# Effect size (MDE / standard deviation)
effect_size = 1000 / std_earnings

# Calculate required sample size for a two-sample t-test (equal-sized groups)
sample_size_per_group = smp.TTestIndPower().solve_power(
    effect_size=effect_size, 
    alpha=0.05, 
    power=0.90, 
    ratio=1,  # Equal-sized treatment and control groups
    alternative='two-sided'
)

# Total sample size needed (treatment + control)
total_sample_size = int(np.ceil(sample_size_per_group * 2))

sample_size_per_group, total_sample_size


(66270.05388214289, 132541)

In [21]:
# Define subgroups: Black men, Black women, Hispanic men, Hispanic women, White/Other men, White/Other women
nlsy97["race_sex"] = nlsy97.apply(lambda row: 
    "Black Men" if row["black"] == 1 and row["female"] == 0 else
    "Black Women" if row["black"] == 1 and row["female"] == 1 else
    "Hispanic Men" if row["hispanic"] == 1 and row["female"] == 0 else
    "Hispanic Women" if row["hispanic"] == 1 and row["female"] == 1 else
    "White Men" if row["black"] == 0 and row["hispanic"] == 0 and row["female"] == 0 else
    "White Women", axis=1)

# Compute standard deviation of earnings by subgroup
subgroup_std = nlsy97.groupby("race_sex")["earnings"].std()

# Perform power analysis for each subgroup
sample_size_results = {}
for group, std in subgroup_std.items():
    effect_size = 1000 / std  # Standardized effect size
    sample_size_per_group = smp.TTestIndPower().solve_power(
        effect_size=effect_size, alpha=0.05, power=0.90, ratio=1, alternative='two-sided'
    )
    total_sample_size = int(np.ceil(sample_size_per_group * 2))
    sample_size_results[group] = {"Std Dev": std, "Per Group": int(np.ceil(sample_size_per_group)), "Total": total_sample_size}

# Convert results to DataFrame and display
sample_size_nlsy97 = pd.DataFrame.from_dict(sample_size_results, orient="index")
sample_size_nlsy97

,Std Dev,Per Group,Total
Black Men,46957.851128,46340,92680
Black Women,40701.499161,34815,69629
Hispanic Men,53780.672437,60784,121567
Hispanic Women,39124.697878,32170,64339
White Men,68653.988955,99052,198104
White Women,51275.386466,55253,110505


## 2.3 Experiment Adjustment

### One-Sided

In [35]:
sample_size_results = {}
for group, std in subgroup_std.items():
    effect_size = 1000 / std  # Standardized effect size
    sample_size_per_group = smp.TTestIndPower().solve_power(
        effect_size=effect_size, alpha=0.05, power=0.90, ratio=1, alternative='larger'
    )
    total_sample_size = int(np.ceil(sample_size_per_group * 2))
    sample_size_results[group] = {"Std Dev": std, "Per Group": int(np.ceil(sample_size_per_group)), "Total": total_sample_size}

# Convert results to DataFrame and display
sample_size_nlsy97 = pd.DataFrame.from_dict(sample_size_results, orient="index")
sample_size_nlsy97

,Std Dev,Per Group,Total
Black Men,46957.851128,37768,75536
Black Women,40701.499161,28375,56750
Hispanic Men,53780.672437,49541,99081
Hispanic Women,39124.697878,26219,52438
White Men,68653.988955,80730,161460
White Women,51275.386466,45033,90065


### Controlled Variables

In [61]:
# Load NLSY97 dataset
nlsy97 = pd.read_pickle(data + 'nlsy97.pkl')

# Rename some columns and then form new dataframe with complete cases for key variables
nlsy97.rename(columns={'avg_earn_2016_to_2020': 'earnings', 'hgc_at_age28': 'yrssch'}, inplace=True)
# nlsy97 = nlsy97[['female','black','hispanic','yrssch','earnings']]
nlsy97 = nlsy97.dropna()

nlsy97["race_sex"] = nlsy97.apply(lambda row: 
    "Black Men" if row["black"] == 1 and row["female"] == 0 else
    "Black Women" if row["black"] == 1 and row["female"] == 1 else
    "Hispanic Men" if row["hispanic"] == 1 and row["female"] == 0 else
    "Hispanic Women" if row["hispanic"] == 1 and row["female"] == 1 else
    "White Men" if row["black"] == 0 and row["hispanic"] == 0 and row["female"] == 0 else
    "White Women", axis=1)

In [ ]:
original_std = nlsy97["earnings"].std()
print(f"Original standard deviation: {original_std:.2f}")

Original standard deviation: 35723.71


In [62]:
import statsmodels.api as sm

# Define dependent variable (earnings) and independent variable (yrssch)
X = sm.add_constant(nlsy97[["yrssch"]])
y = nlsy97["earnings"]

# Fit regression model
model = sm.OLS(y, X).fit()

# Compute residual standard deviation
residual_std = model.resid.std()
print(f"Residual standard deviation after controlling for education: {residual_std:.2f}")

import statsmodels.stats.power as smp

# Parameters
alpha = 0.05  # Significance level
power = 0.90  # Required power
effect_size = 1000 / residual_std  # Standardized effect size

# Compute new sample size after controlling for education
new_sample_size = smp.TTestIndPower().solve_power(
    effect_size=effect_size, alpha=alpha, power=power, ratio=1, alternative='two-sided'
)

print(f"New required sample size per group after controlling for education: {new_sample_size:.0f}")

Residual standard deviation after controlling for education: 31796.07
New required sample size per group after controlling for education: 21247


### Bootstrap Increase Sample Size

In [63]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import ttest_ind
from statsmodels.stats.power import TTestIndPower
from linearmodels.iv import IV2SLS

# ✅ Step 1: Rename columns for clarity and create a cleaned dataset
nlsy97.rename(columns={'avg_earn_2016_to_2020': 'earnings', 'hgc_at_age28': 'yrssch'}, inplace=True)
nlsy97_cleaned = nlsy97[['female','black','hispanic','yrssch','earnings']].dropna()

# ✅ Step 2: Run OLS to control for education and compute residual standard deviation
X = sm.add_constant(nlsy97_cleaned[["yrssch"]])  
y = nlsy97_cleaned["earnings"]
model = sm.OLS(y, X).fit()

# Compute residual standard deviation after controlling for education
residual_std = model.resid.std()
print(f"Residual standard deviation after controlling for education: {residual_std:.2f}")

# ✅ Step 3: Determine the required sample size (Power = 90%)
earnings_col = "earnings"
analysis = TTestIndPower()
effect_size = 1000 / residual_std  # Updated effect size after controlling for education

required_sample_size = int(np.ceil(analysis.solve_power(effect_size=effect_size, alpha=0.05, power=0.90, ratio=1.0)))

# ✅ Step 4: Bootstrap sampling (Increase sample size)
bootstrap_nlsy97 = nlsy97_cleaned.sample(n=max(required_sample_size, 120000), replace=True, random_state=42)

# ✅ Step 5: Apply stratified randomization (balances across demographics)
np.random.seed(42)
bootstrap_nlsy97["job_training"] = bootstrap_nlsy97.groupby(["female", "black", "hispanic", "yrssch"])[earnings_col].transform(
    lambda x: np.random.choice([0, 1], size=len(x), p=[0.5, 0.5])
)

# ✅ Step 6: Adjust the treatment effect (ATE) to ensure a $1,000 increase
mean_control = bootstrap_nlsy97[bootstrap_nlsy97["job_training"] == 0][earnings_col].mean()
mean_treatment = bootstrap_nlsy97[bootstrap_nlsy97["job_training"] == 1][earnings_col].mean()

# Adjustment to enforce a $1,000 ATE
adjustment = (mean_control + 1000 - mean_treatment) * 1.05  
print(f"Adjustment Value: {adjustment}")

bootstrap_nlsy97.loc[bootstrap_nlsy97["job_training"] == 1, earnings_col] += adjustment

# ✅ Step 7: Perform a t-test to validate the adjusted ATE
t_stat, p_value = ttest_ind(
    bootstrap_nlsy97[bootstrap_nlsy97["job_training"] == 1][earnings_col],
    bootstrap_nlsy97[bootstrap_nlsy97["job_training"] == 0][earnings_col],
    equal_var=False
)

# ✅ Step 8: Difference-in-Differences (DiD) analysis
bootstrap_nlsy97["post_treatment"] = (bootstrap_nlsy97["yrssch"] - bootstrap_nlsy97["yrssch"].min()) / bootstrap_nlsy97["yrssch"].std()
bootstrap_nlsy97["interaction"] = bootstrap_nlsy97["job_training"] * bootstrap_nlsy97["post_treatment"]

X = sm.add_constant(bootstrap_nlsy97[["job_training", "post_treatment", "interaction"]])
y = bootstrap_nlsy97[earnings_col]
did_model = sm.OLS(y, X).fit()

# ✅ Step 9: LATE (Instrumental Variable) analysis
bootstrap_nlsy97["treatment_received"] = np.where(np.random.rand(len(bootstrap_nlsy97)) > 0.1, bootstrap_nlsy97["job_training"], 0)

iv_model = IV2SLS.from_formula(
    f"{earnings_col} ~ 1 + [treatment_received ~ job_training]", 
    data=bootstrap_nlsy97
).fit()

# ✅ Step 10: Compute statistical power after adjustments
actual_power = analysis.solve_power(effect_size=effect_size, nobs1=len(bootstrap_nlsy97)//2, alpha=0.05, power=None, ratio=1.0)

# ✅ Display results
print("\n🔹 Sample Size Adjustment Results 🔹")
print(f"Final Sample Size: {len(bootstrap_nlsy97)} (Target: {required_sample_size})")
print(f"T-test Statistic: {t_stat:.4f}, P-value: {p_value:.4f}")
print(f"DiD Analysis Result (Interaction Coeff, ATE): {did_model.params['interaction']:.2f}")
print(f"LATE Analysis Result (ATE Adjustment): {iv_model.params['treatment_received']:.2f}")
print(f"Statistical Power: {actual_power:.2f}")


Residual standard deviation after controlling for education: 31796.07
Adjustment Value: 902.419758885062

🔹 Sample Size Adjustment Results 🔹
Final Sample Size: 120000 (Target: 21247)
T-test Statistic: 5.1777, P-value: 0.0000
DiD Analysis Result (Interaction Coeff, ATE): 77.00
LATE Analysis Result (ATE Adjustment): 1159.67
Statistical Power: 1.00
